In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error

from IPython.display import clear_output
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!pip install autoviz
!pip install xlrd

from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()

clear_output()

In [ ]:
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

# AUTO EDA

In [ ]:
ff = AV.AutoViz('../input/30-days-of-ml/train.csv')

In [ ]:
!nvidia-smi

# MODELING

In [ ]:
from xgboost import XGBRegressor
xgb_params = {
    'n_estimators': 5000,
    'learning_rate': 0.12,
    'subsample': 0.96,
    'colsample_bytree': 0.12,
    'max_depth': 2,
    'booster': 'gbtree', 
    'reg_lambda': 65.1,
    'reg_alpha': 15.9,
    'random_state':40
}


In [ ]:
df = pd.read_csv("../input/30days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

# KFOLDS

This dataset is the same as abishek thakur created!

In [ ]:
final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    params ={'learning_rate': 0.12048476513286424,
 'reg_lambda': 2.7877111396373698e-08,
 'reg_alpha': 0.0055110804060212225,
 'subsample': 0.6126668712919948,
 'colsample_bytree': 0.8444075670111965,
 'max_depth': 2}
    
    model = XGBRegressor(
        random_state=0, 
        tree_method='gpu_hist',
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=5000,
        **params
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

# PREDICTIONS

In [ ]:
preds = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.target = preds
sample_submission.to_csv("xgb_submission_k", index=False)

sample_submission.head()